In [30]:
"""
txhex:
020000000001017bfd0d2548b1809824d3b2d1aa37e33de8d73a18a5c254a6ba0f08f6dc5b16120100000000feffffff026ef3240100000000160014561223656dc1b82d064d2207c1917222d7c06586a086010000000000160014b53648c0b44805c21727eb7b23e5441ce51867610247304402200e62dc69511d09a1a8e11efe282ad5c010c9473c5b7754661689dfcc6afaeb96022070ed98d3409e495033652612d7dbf8fbd3051446d78fc19b640cecb154f3ae8f012102bca743b84b1705f56d3b170b22f7331758d31ff722d04c5192e8918fe692b5cabe212000
----------
02000000 - version
00 - segwit marker
01 - segwit flag
01 - # of inputs
7bfd0d2548b1809824d3b2d1aa37e33de8d73a18a5c254a6ba0f08f6dc5b1612 - previous tx hash
01000000 - privous tx index
00 - ScriptSig
feffffff - sequence
02 - # of outputs
6ef3240100000000 - output amounts
160014561223656dc1b82d064d2207c1917222d7c06586 - script pub key
a086010000000000 - output amounts
160014b53648c0b44805c21727eb7b23e5441ce5186761 - script pub key
0247304402200e62dc69511d09a1a8e11efe282ad5c010c9473c5b7754661689dfcc6afaeb96022070ed98d3409e495033652612d7dbf8fbd3051446d78fc19b640cecb154f3ae8f012102bca743b84b1705f56d3b170b22f7331758d31ff722d04c5192e8918fe692b5ca
be212000 - locktime
"""

from io import BytesIO
from unittest import TestCase

import json
import requests

from ecc import PrivateKey
from helper import (
    encode_varint,
    hash256,
    int_to_little_endian,
    little_endian_to_int,
    read_varint,
    SIGHASH_ALL,
)
from script import p2pkh_script, Script


class TxFetcher:
    cache = {}

    @classmethod
    def get_url(cls, testnet=True):
        if testnet:
            return 'https://blockstream.info/testnet/api'
        else:
            return 'https://blockstream.info/api'

    @classmethod
    def fetch(cls, tx_id, testnet=True, fresh=False):
        if fresh or (tx_id not in cls.cache):
            url = '{}/tx/{}/hex'.format(cls.get_url(testnet), tx_id)
            response = requests.get(url)
            try:
                raw = bytes.fromhex(response.text.strip())
            except ValueError:
                raise ValueError('unexpected response: {}'.format(response.text))
            tx = Tx.parse(BytesIO(raw), testnet=testnet)
            # make sure the tx we got matches to the hash we requested
            if tx.segwit:
                computed = tx.id()
            else:
                computed = hash256(raw)[::-1].hex()
            if computed != tx_id:
                raise RuntimeError('server lied: {} vs {}'.format(computed, tx_id))
            cls.cache[tx_id] = tx
        cls.cache[tx_id].testnet = testnet
        return cls.cache[tx_id]

    @classmethod
    def load_cache(cls, filename):
        disk_cache = json.loads(open(filename, 'r').read())
        for k, raw_hex in disk_cache.items():
            cls.cache[k] = Tx.parse(BytesIO(bytes.fromhex(raw_hex)))

    @classmethod
    def dump_cache(cls, filename):
        with open(filename, 'w') as f:
            to_dump = {k: tx.serialize().hex() for k, tx in cls.cache.items()}
            s = json.dumps(to_dump, sort_keys=True, indent=4)
            f.write(s)


class TxIn:

    def __init__(self, prev_tx, prev_index, script_sig=None, sequence=0xffffffff):
        self.prev_tx = prev_tx
        self.prev_index = prev_index
        if script_sig is None:
            self.script_sig = Script()
        else:
            self.script_sig = script_sig
        self.sequence = sequence

    def __repr__(self):
        return '{}:{}'.format(
            self.prev_tx.hex(),
            self.prev_index,
        )

    @classmethod
    def parse(cls, s):
        '''Takes a byte stream and parses the tx_input at the start
        return a TxIn object
        '''
        # reverse the little endian bytes
        prev_tx = s.read(32)[::-1]
        # prev_index is an integer in 4 bytes, little endian
        prev_index = little_endian_to_int(s.read(4))
        # use Script.parse to get the ScriptSig
        script_sig = Script.parse(s)
        # sequence is an integer in 4 bytes, little-endian
        sequence = little_endian_to_int(s.read(4))
        # return an instance of the class (see __init__ for args)
        return cls(prev_tx, prev_index, script_sig, sequence)

    def serialize(self):
        """Returns the byte serialization of the transaction input"""
        # serialize prev_tx, little endian
        result = self.prev_tx[::-1]
        # serialize prev_index, 4 bytes, little endian
        result += int_to_little_endian(self.prev_index, 4)
        # serialize the script_sig
        result += self.script_sig.serialize()
        # serialize sequence, 4 bytes, little endian
        result += int_to_little_endian(self.sequence, 4)
        return result

    def fetch_tx(self, testnet=True):
        return TxFetcher.fetch(self.prev_tx.hex(), testnet=testnet)

    def value(self, testnet=True):
        """Get the outpoint value by looking up the tx hash
        Returns the amount in satoshi
        """
        # use self.fetch_tx to get the transaction
        tx = self.fetch_tx(testnet=testnet)
        # get the output at self.prev_index
        # return the amount property
        return tx.tx_outs[self.prev_index].amount

    def script_pubkey(self, testnet=True):
        """Get the ScriptPubKey by looking up the tx hash
        Returns a Script object
        """
        # use self.fetch_tx to get the transaction
        tx = self.fetch_tx(testnet=testnet)
        # get the output at self.prev_index
        # return the script_pubkey property
        return tx.tx_outs[self.prev_index].script_pubkey


class TxOut:

    def __init__(self, amount, script_pubkey):
        self.amount = amount
        self.script_pubkey = script_pubkey

    def __repr__(self):
        return '{}:{}'.format(self.amount, self.script_pubkey)

    @classmethod
    def parse(cls, s):
        '''Takes a byte stream and parses the tx_output at the start
        return a TxOut object
        '''
        # amount is an integer in 8 bytes, little endian
        amount = little_endian_to_int(s.read(8))
        # use Script.parse to get the ScriptPubKey
        script_pubkey = Script.parse(s)
        # return an instance of the class (see __init__ for args)
        return cls(amount, script_pubkey)

    def serialize(self):
        '''Returns the byte serialization of the transaction output'''
        # serialize amount, 8 bytes, little endian
        result = int_to_little_endian(self.amount, 8)
        # serialize the script_pubkey
        result += self.script_pubkey.serialize()
        return result


class Tx:
    command = b'tx'

    def __init__(self, version, tx_ins, tx_outs, locktime, testnet=True, segwit=False):
        self.version = version
        self.tx_ins = tx_ins
        self.tx_outs = tx_outs
        self.testnet = testnet
        self.locktime = locktime
        self.segwit = segwit

    @classmethod
    def parse(cls, s, testnet=True):
        s.read(4)
        # the fifth bytes tells whether it's a segwit transaction
        if s.read(1) == b'\x00':
            parse_method = cls.parse_segwit
        else:
            parse_method = cls.parse_legacy
        # move the cursor back the the beginning of the transaction stream
        s.seek(-5, 1)
        return parse_method(s, testnet=testnet)

    @classmethod
    def parse_legacy(cls, s, testnet=True):
        version = little_endian_to_int(s.read(4))  # <4>
        num_inputs = read_varint(s)
        inputs = []
        for _ in range(num_inputs):
            inputs.append(TxIn.parse(s))
        num_outputs = read_varint(s)
        outputs = []
        for _ in range(num_outputs):
            outputs.append(TxOut.parse(s))
        locktime = little_endian_to_int(s.read(4))
        return cls(version, inputs, outputs, locktime,
                   testnet=testnet, segwit=False)

    @classmethod
    def parse_segwit(cls, s, testnet=True):
        version = little_endian_to_int(s.read(4))
        marker = s.read(2)
        if marker != b'\x00\x01':  # <1>
            raise RuntimeError('Not a segwit transaction {}'.format(marker))
        num_inputs = read_varint(s)
        inputs = []
        for idx in range(num_inputs):
            inputs.append(TxIn.parse(s))
        num_outputs = read_varint(s)
        outputs = []
        for _ in range(num_outputs):
            outputs.append(TxOut.parse(s))
        for tx_in in inputs:  # <2>
            num_items = read_varint(s)
            items = []
            for _ in range(num_items):
                item_len = read_varint(s)
                if item_len == 0:
                    items.append(0)
                else:
                    items.append(s.read(item_len))
            tx_in.witness = items
        locktime = little_endian_to_int(s.read(4))
        return cls(version, inputs, outputs, locktime,
                   testnet=testnet, segwit=True)

In [31]:
raw_tx=bytes.fromhex("020000000001017bfd0d2548b1809824d3b2d1aa37e33de8d73a18a5c254a6ba0f08f6dc5b16120100000000feffffff026ef3240100000000160014561223656dc1b82d064d2207c1917222d7c06586a086010000000000160014b53648c0b44805c21727eb7b23e5441ce51867610247304402200e62dc69511d09a1a8e11efe282ad5c010c9473c5b7754661689dfcc6afaeb96022070ed98d3409e495033652612d7dbf8fbd3051446d78fc19b640cecb154f3ae8f012102bca743b84b1705f56d3b170b22f7331758d31ff722d04c5192e8918fe692b5cabe212000")
stream = BytesIO(raw_tx)
tx = Tx.parse(stream)
print("version: {}, locktime: {}, txins: {}, txouts {}".format(tx.version, tx.locktime, tx.tx_ins, tx.tx_outs))



parse segwit input 0
version: 2, locktime: 2105790, txins: [12165bdcf6080fbaa654c2a5183ad7e83de337aad1b2d3249880b148250dfd7b:1], txouts [19198830:OP_0 561223656dc1b82d064d2207c1917222d7c06586, 100000:OP_0 b53648c0b44805c21727eb7b23e5441ce5186761]
